In [1]:
import json
import os
from pyspark.ml import Pipeline
from sparknlp.base import *
from sparknlp.annotator import *
import sparknlp
import pandas as pd
import openpyxl
import numpy as np
spark = sparknlp.start()

def get_ann_pipeline ():
    
    document_assembler = DocumentAssembler() \
        .setInputCol("text")\
        .setOutputCol('document')

    sentence = SentenceDetector()\
        .setInputCols(['document'])\
        .setOutputCol('sentence')\
        .setCustomBounds(['\n'])

    tokenizer = Tokenizer() \
        .setInputCols(["sentence"]) \
        .setOutputCol("token")

    pos = PerceptronModel.pretrained() \
              .setInputCols(["sentence", "token"]) \
              .setOutputCol("pos")
    
    embeddings = WordEmbeddingsModel.pretrained()\
          .setInputCols(["sentence", "token"])\
          .setOutputCol("embeddings")

    ner_model = NerDLModel.load("outputs/ner_wiki_glove100d_en")\
       .setInputCols(["sentence", "token", "embeddings"])\
       .setOutputCol("ner")

    ner_converter = NerConverter()\
      .setInputCols(["sentence", "token", "ner"])\
      .setOutputCol("ner_chunk")

    ner_pipeline = Pipeline(
        stages = [
            document_assembler,
            sentence,
            tokenizer,
            pos,
            embeddings,
            ner_model,
            ner_converter
        ]
    )

    empty_data = spark.createDataFrame([[""]]).toDF("text")

    ner_pipelineFit = ner_pipeline.fit(empty_data)

    ner_lp_pipeline = LightPipeline(ner_pipelineFit)
    print ("Spark NLP NER lightpipeline is created")

    return ner_lp_pipeline

23/01/20 09:53:22 WARN Utils: Your hostname, Vader resolves to a loopback address: 127.0.1.1; using 192.168.1.94 instead (on interface enp5s0f0)
23/01/20 09:53:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/sjhjrok/anaconda3/envs/NLP/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sjhjrok/.ivy2/cache
The jars for the packages stored in: /home/sjhjrok/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-83040d61-bb8e-4b0a-be60-1bb1c3781ca4;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#l

23/01/20 09:53:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/20 09:53:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
conll_pipeline = get_ann_pipeline ()

pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[ | ]pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[ / ]Download done! Loading the resource.
[ — ]

[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ]glove_100d download started this may take some time.
Approximate size to download 145.3 MB
Download done! Loading the resource.
[OK!]


2023-01-20 09:53:43.012276: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-20 09:53:43.079592: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_nam

Spark NLP NER lightpipeline is created


In [3]:
parsed = conll_pipeline.annotate ("Where am I")
parsed

sentances  = ["i want to make coffee"]
sentances.append("where am i")
sentances.append("what can i do here")
sentances.append("prepare meal")
sentances.append("use refrigerator")
sentances.append("use fan")
sentances.append("use oven")
sentances.append("use stove")
sentances.append("i would like to wash sheets")
sentances.append("i would like to watch television")
sentances.append("wash sheets")
sentances.append("wash clothes")
sentances.append("cook dinner")
sentances.append("i would like to cook dinner")
sentances.append("where is the coffee")
sentances.append("rewind video")
sentances.append("pause video")
sentances.append("skip forward")
sentances.append("negative")
sentances.append("positive")
sentances.append("yes")
sentances.append("no")
sentances.append("hello yes")
sentances.append("hello no")
sentances.append("shutdown app")
sentances.append("shut")
sentances.append("hello")
sentances.append("play video")
sentances.append("hello shut down app")
sentances.append("hello shut")
sentances.append("hello play video")
sentances.append("hello negative")
sentances.append("hello positive")
sentances.append("correct")
sentances.append("incorrect")
sentances.append("hello correct")
sentances.append("hello incorrect")
sentances.append("i want to make coffee")
sentances.append("where am i")


df = pd.read_excel("./Keyword Map NLP.xlsx")
Loc = df.loc[df['Loc']==1]
#print(Loc)
workbook = openpyxl.load_workbook('./Keyword Map NLP.xlsx')
sh = workbook.active
                 
get= []
print("start")
for i in range(1,sh.max_column+1):
    #print(i)
    search = sh.cell(row=1,column=i).value
    #print(search)
    if(search == 'Loc'):
        for j in range(2,sh.max_row+1):
            one = sh.cell(row=j,column=i).value
            if(one == 1):
                #print(sh.cell(row=j,column=1).value)
                get.append(sh.cell(row=j,column=1).value)
        break
                 
for w in get:
    filer = (f"i want to go to the {w}")
    sentances.append(filer)

for w in get:
    filer = (f"navigate to {w}")
    sentances.append(filer)
                 
for i in range(1,sh.max_row+1):
    keyword = sh.cell(row=i,column=1).value
    sentances.append(keyword)                          


with open('check.train', 'w') as file:
      

    file.writelines("% s\n" % data for data in sentances)

start


In [4]:
store = ["-DOCSTART- -X- -X- O\n"]

conll_lines=''

for l in sentances:
    parsed = conll_pipeline.annotate(l)
    for token, pos, ner in zip(parsed['token'],parsed['pos'],parsed['ner']):

        conll_lines += "{} {} {} {}\n".format(token, pos, pos, ner)
        #print(conll_lines)
        
    conll_lines+="\n"
    #print(conll_lines)
store.append(conll_lines)


with open('eng_custom_bert.train', 'w') as file:
      

    file.writelines("% s\n" % data for data in store)


In [5]:
sentances  = ["hello i want to make coffee"]
for w in get:
    filer = (f"hello i want to go to the {w}")
    sentances.append(filer)
for w in get:
    filer = (f"hello navigate to {w}")
    sentances.append(filer)

#filer = (f"hello i want to make coffee")
#sentances.append(filer)
filer = (f"hello where am i")
sentances.append(filer)
filer = (f"hello what can i do here")
sentances.append(filer)
filer = (f"hello prepare meal")
sentances.append(filer)
filer = (f"hello use refrigerator")
sentances.append(filer)
filer = (f"hello start camera")
sentances.append(filer)

filer = (f"hello correct")
sentances.append(filer)
filer = (f"hello incorrect")
sentances.append(filer)
filer = (f"hello play video")
sentances.append(filer)
filer = (f"hello negative")
sentances.append(filer)
filer = (f"hello positive")
sentances.append(filer)


filer = (f"hello yes")
sentances.append(filer)
filer = (f"hello no")
sentances.append(filer)
filer = (f"hello location")
sentances.append(filer)
filer = (f"hello location.")
sentances.append(filer)
filer = (f"hello activity")
sentances.append(filer)
with open('check.testa', 'w') as file:
      

    file.writelines("% s\n" % data for data in sentances)

In [6]:
store = ["-DOCSTART- -X- -X- O\n"]

conll_lines=''

for l in sentances:
    parsed = conll_pipeline.annotate (l)
    for token, pos, ner in zip(parsed['token'],parsed['pos'],parsed['ner']):

        conll_lines += "{} {} {} {}\n".format(token, pos, pos, ner)
        
    conll_lines+="\n"
store.append(conll_lines)


with open('eng_custom_bert.testa', 'w') as file:
      

    file.writelines("% s\n" % data for data in store)

In [7]:
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *

from sparknlp.common import *
from sparknlp.pretrained import PretrainedPipeline

from sparknlp.training import CoNLL
training_data = CoNLL().readDataset(spark, './eng_custom_bert.train')
training_data.show()
testing_data= CoNLL().readDataset(spark, './eng_custom_bert.testa')

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|i want to make co...|[{document, 0, 20...|[{document, 0, 20...|[{token, 0, 0, i,...|[{pos, 0, 0, NNP,...|[{named_entity, 0...|
|          where am i|[{document, 0, 9,...|[{document, 0, 9,...|[{token, 0, 4, wh...|[{pos, 0, 4, WRB,...|[{named_entity, 0...|
|  what can i do here|[{document, 0, 17...|[{document, 0, 17...|[{token, 0, 3, wh...|[{pos, 0, 3, WP, ...|[{named_entity, 0...|
|        prepare meal|[{document, 0, 11...|[{document, 0, 11...|[{token, 0, 6, pr...|[{pos, 0, 6, VB, ...|[{named_entity, 0...|
|    use refrigerator|[{document, 0, 15...|[{document, 0, 15...|[{token, 0, 2, us...|[{pos, 0, 2, NN, ..

In [8]:
bert = BertEmbeddings.pretrained('bert_base_cased', 'en')\
    .setInputCols(["sentence",'token'])\
    .setOutputCol("bert")\
    .setCaseSensitive(False)



nerTagger = NerDLApproach()\
    .setInputCols(["sentence", "token", "bert"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(200)\
    .setLr(0.003)\
    .setBatchSize(32)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setValidationSplit(0.2)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setOutputLogsPath('ner_logs')\
    .setTestDataset("test_withEmbeds.parquet")


test_data = CoNLL().readDataset(spark, './eng_custom_bert.testa')
test_data = bert.transform(test_data)

import os
 
# Directory name
directory = "test_withEmbeds.parquet"
 
# Parent Directory
parent = "/home/sjhjrok/Documents/NLP/"
 
# Path
path = os.path.join(parent, directory)
 
# Remove the Directory
# "GeeksforGeeks"
try:
    os.rmdir(path)
    print("Directory '% s' has been removed successfully" % directory)
    test_data.write.parquet("test_withEmbeds.parquet")
except OSError as error:
    print(error)
    print("Directory '% s' can not be removed" % directory)





ner_pipeline = Pipeline(stages = [bert, nerTagger])
ner_model = ner_pipeline.fit(training_data)

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[ | ]bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[ / ]Download done! Loading the resource.
[OK!]
[Errno 2] No such file or directory: '/home/sjhjrok/Documents/NLP/test_withEmbeds.parquet'
Directory 'test_withEmbeds.parquet' can not be removed


2023-01-20 09:54:16.117753: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

Training started - total epochs: 200 - lr: 0.003 - batch size: 32 - labels: 9 - chars: 22 - training examples: 198
Epoch 1/200 started, lr: 0.003, dataset size: 198
Epoch 1/200 - 0.73s - loss: 29.962303 - batches: 7
Quality on validation dataset (20.0%), validation examples = 39
time to finish evaluation: 0.18s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 9	 2	 1	 0.8181818	 0.9	 0.8571428
Loc	 16	 1	 5	 0.9411765	 0.7619048	 0.84210527
Act	 29	 20	 0	 0.59183675	 1.0	 0.74358976
Neg	 0	 0	 3	 0.0	 0.0	 0.0
0	 42	 0	 3	 1.0	 0.93333334	 0.9655172
Obj	 0	 0	 8	 0.0	 0.0	 0.0
Pos	 0	 0	 1	 0.0	 0.0	 0.0
Trigger	 0	 0	 2	 0.0	 0.0	 0.0
tp: 96 fp: 23 fn: 23 labels: 8
Macro-average	 prec: 0.4188994, rec: 0.44940478, f1: 0.43361622
Micro-average	 prec: 0.8067227, rec: 0.8067227, f1: 0.8067227
Quality on test dataset: 
time to finish evaluation: 0.02s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 30	 33	 0	 0.47619048	 1.0	 0.64516133
Loc	 53	 1	 5	 0.9814815	 0.9137931	 0.94642854
Act	 56	 51	 3	 0.5233645

Epoch 9/200 - 0.12s - loss: 4.48909 - batches: 7
Quality on validation dataset (20.0%), validation examples = 39
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 1	 0	 0.90909094	 1.0	 0.95238096
Loc	 18	 0	 3	 1.0	 0.85714287	 0.9230769
Act	 28	 4	 1	 0.875	 0.9655172	 0.91803277
Neg	 0	 0	 3	 0.0	 0.0	 0.0
0	 44	 1	 1	 0.9777778	 0.9777778	 0.9777778
Obj	 7	 2	 1	 0.7777778	 0.875	 0.8235294
Pos	 0	 2	 1	 0.0	 0.0	 0.0
Trigger	 2	 0	 0	 1.0	 1.0	 1.0
tp: 109 fp: 10 fn: 10 labels: 8
Macro-average	 prec: 0.6924558, rec: 0.70942974, f1: 0.70084006
Micro-average	 prec: 0.9159664, rec: 0.9159664, f1: 0.9159664
Quality on test dataset: 
time to finish evaluation: 0.02s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 30	 29	 0	 0.5084746	 1.0	 0.6741573
Loc	 56	 0	 2	 1.0	 0.9655172	 0.9824561
Act	 58	 12	 1	 0.82857144	 0.9830508	 0.89922476
Neg	 0	 0	 3	 0.0	 0.0	 0.0
0	 135	 0	 0	 1.0	 1.0	 1.0
Obj	 5	 1	 0	 0.8333333	 1.0	 0.90909094
Pos	 3	 3	 0	 0.5	 1.0	 0.6666667
Tri

Epoch 16/200 - 0.12s - loss: 1.6912378 - batches: 7
Quality on validation dataset (20.0%), validation examples = 39
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 19	 0	 2	 1.0	 0.9047619	 0.95
Act	 26	 2	 3	 0.9285714	 0.8965517	 0.91228074
Neg	 1	 0	 2	 1.0	 0.33333334	 0.5
0	 44	 1	 1	 0.9777778	 0.9777778	 0.9777778
Obj	 7	 4	 1	 0.6363636	 0.875	 0.7368421
Pos	 1	 2	 0	 0.33333334	 1.0	 0.5
Trigger	 2	 0	 0	 1.0	 1.0	 1.0
tp: 110 fp: 9 fn: 9 labels: 8
Macro-average	 prec: 0.8595058, rec: 0.8734281, f1: 0.86641103
Micro-average	 prec: 0.92436975, rec: 0.92436975, f1: 0.92436975
Quality on test dataset: 
time to finish evaluation: 0.02s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 30	 9	 0	 0.7692308	 1.0	 0.8695652
Loc	 57	 0	 1	 1.0	 0.98275864	 0.99130434
Act	 59	 4	 0	 0.93650794	 1.0	 0.9672131
Neg	 1	 0	 2	 1.0	 0.33333334	 0.5
0	 135	 0	 0	 1.0	 1.0	 1.0
Obj	 5	 0	 0	 1.0	 1.0	 1.0
Pos	 3	 2	 0	 0.6	 1.0	 0.75
Trigger	 55	 1	 13	 

...

Epoch 168/200 - 0.11s - loss: 0.1262658 - batches: 7
Quality on validation dataset (20.0%), validation examples = 39
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 20	 0	 1	 1.0	 0.95238096	 0.9756098
Act	 25	 1	 4	 0.96153843	 0.86206895	 0.9090909
Neg	 3	 0	 0	 1.0	 1.0	 1.0
0	 45	 1	 0	 0.9782609	 1.0	 0.989011
Obj	 7	 4	 1	 0.6363636	 0.875	 0.7368421
Pos	 1	 0	 0	 1.0	 1.0	 1.0
Trigger	 2	 0	 0	 1.0	 1.0	 1.0
tp: 113 fp: 6 fn: 6 labels: 8
Macro-average	 prec: 0.94702035, rec: 0.9611812, f1: 0.9540482
Micro-average	 prec: 0.94957983, rec: 0.94957983, f1: 0.94957983
Quality on test dataset: 
time to finish evaluation: 0.02s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 30	 0	 0	 1.0	 1.0	 1.0
Loc	 57	 0	 1	 1.0	 0.98275864	 0.99130434
Act	 59	 0	 0	 1.0	 1.0	 1.0
Neg	 3	 0	 0	 1.0	 1.0	 1.0
0	 135	 0	 0	 1.0	 1.0	 1.0
Obj	 5	 0	 0	 1.0	 1.0	 1.0
Pos	 3	 0	 0	 1.0	 1.0	 1.0
Trigger	 68	 1	 0	 0.98550725	 1.0	 0.99270076
tp: 360 fp: 1 fn: 1

Epoch 176/200 - 0.12s - loss: 0.08854623 - batches: 7
Quality on validation dataset (20.0%), validation examples = 39
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 20	 0	 1	 1.0	 0.95238096	 0.9756098
Act	 25	 1	 4	 0.96153843	 0.86206895	 0.9090909
Neg	 2	 0	 1	 1.0	 0.6666667	 0.8
0	 45	 1	 0	 0.9782609	 1.0	 0.989011
Obj	 7	 4	 1	 0.6363636	 0.875	 0.7368421
Pos	 1	 1	 0	 0.5	 1.0	 0.6666667
Trigger	 2	 0	 0	 1.0	 1.0	 1.0
tp: 112 fp: 7 fn: 7 labels: 8
Macro-average	 prec: 0.88452035, rec: 0.91951454, f1: 0.901678
Micro-average	 prec: 0.9411765, rec: 0.9411765, f1: 0.9411765
Quality on test dataset: 
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 30	 0	 0	 1.0	 1.0	 1.0
Loc	 57	 0	 1	 1.0	 0.98275864	 0.99130434
Act	 59	 0	 0	 1.0	 1.0	 1.0
Neg	 2	 0	 1	 1.0	 0.6666667	 0.8
0	 135	 0	 0	 1.0	 1.0	 1.0
Obj	 5	 0	 0	 1.0	 1.0	 1.0
Pos	 3	 1	 0	 0.75	 1.0	 0.85714287
Trigger	 68	 1	 0	 0.98550725	 1.0	 0.9927

Epoch 184/200 - 0.12s - loss: 0.24499823 - batches: 7
Quality on validation dataset (20.0%), validation examples = 39
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 20	 2	 1	 0.90909094	 0.95238096	 0.9302326
Act	 26	 1	 3	 0.962963	 0.8965517	 0.9285714
Neg	 3	 0	 0	 1.0	 1.0	 1.0
0	 45	 1	 0	 0.9782609	 1.0	 0.989011
Obj	 6	 2	 2	 0.75	 0.75	 0.75
Pos	 1	 0	 0	 1.0	 1.0	 1.0
Trigger	 2	 0	 0	 1.0	 1.0	 1.0
tp: 113 fp: 6 fn: 6 labels: 8
Macro-average	 prec: 0.9500394, rec: 0.9498666, f1: 0.949953
Micro-average	 prec: 0.94957983, rec: 0.94957983, f1: 0.94957983
Quality on test dataset: 
time to finish evaluation: 0.02s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 30	 0	 0	 1.0	 1.0	 1.0
Loc	 57	 0	 1	 1.0	 0.98275864	 0.99130434
Act	 59	 0	 0	 1.0	 1.0	 1.0
Neg	 3	 0	 0	 1.0	 1.0	 1.0
0	 135	 0	 0	 1.0	 1.0	 1.0
Obj	 5	 0	 0	 1.0	 1.0	 1.0
Pos	 3	 0	 0	 1.0	 1.0	 1.0
Trigger	 68	 1	 0	 0.98550725	 1.0	 0.99270076
tp: 360 fp: 1 fn: 1 labels:

Epoch 192/200 - 0.12s - loss: 0.0912981 - batches: 7
Quality on validation dataset (20.0%), validation examples = 39
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 20	 0	 1	 1.0	 0.95238096	 0.9756098
Act	 26	 1	 3	 0.962963	 0.8965517	 0.9285714
Neg	 3	 0	 0	 1.0	 1.0	 1.0
0	 45	 1	 0	 0.9782609	 1.0	 0.989011
Obj	 7	 3	 1	 0.7	 0.875	 0.7777778
Pos	 1	 0	 0	 1.0	 1.0	 1.0
Trigger	 2	 0	 0	 1.0	 1.0	 1.0
tp: 114 fp: 5 fn: 5 labels: 8
Macro-average	 prec: 0.955153, rec: 0.9654916, f1: 0.9602945
Micro-average	 prec: 0.9579832, rec: 0.9579832, f1: 0.9579832
Quality on test dataset: 
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 30	 0	 0	 1.0	 1.0	 1.0
Loc	 58	 0	 0	 1.0	 1.0	 1.0
Act	 59	 0	 0	 1.0	 1.0	 1.0
Neg	 3	 0	 0	 1.0	 1.0	 1.0
0	 135	 0	 0	 1.0	 1.0	 1.0
Obj	 5	 0	 0	 1.0	 1.0	 1.0
Pos	 3	 0	 0	 1.0	 1.0	 1.0
Trigger	 68	 0	 0	 1.0	 1.0	 1.0
tp: 361 fp: 0 fn: 0 labels: 8
Macro-average	 prec: 1.0, rec: 

Epoch 200/200 - 0.12s - loss: 0.102068394 - batches: 7
Quality on validation dataset (20.0%), validation examples = 39
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 10	 0	 0	 1.0	 1.0	 1.0
Loc	 20	 0	 1	 1.0	 0.95238096	 0.9756098
Act	 25	 1	 4	 0.96153843	 0.86206895	 0.9090909
Neg	 3	 0	 0	 1.0	 1.0	 1.0
0	 45	 1	 0	 0.9782609	 1.0	 0.989011
Obj	 7	 4	 1	 0.6363636	 0.875	 0.7368421
Pos	 1	 0	 0	 1.0	 1.0	 1.0
Trigger	 2	 0	 0	 1.0	 1.0	 1.0
tp: 113 fp: 6 fn: 6 labels: 8
Macro-average	 prec: 0.94702035, rec: 0.9611812, f1: 0.9540482
Micro-average	 prec: 0.94957983, rec: 0.94957983, f1: 0.94957983
Quality on test dataset: 
time to finish evaluation: 0.02s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 30	 0	 0	 1.0	 1.0	 1.0
Loc	 56	 0	 2	 1.0	 0.9655172	 0.9824561
Act	 59	 0	 0	 1.0	 1.0	 1.0
Neg	 3	 0	 0	 1.0	 1.0	 1.0
0	 135	 0	 0	 1.0	 1.0	 1.0
Obj	 5	 0	 0	 1.0	 1.0	 1.0
Pos	 3	 0	 0	 1.0	 1.0	 1.0
Trigger	 68	 2	 0	 0.9714286	 1.0	 0.9855072
tp: 359 fp: 2 fn: 2 l

In [9]:
ner_model.stages[1].write().overwrite().save('NER_bert_2023_01')

2023-01-20 09:54:48.004930: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

In [10]:
# You can use any word embeddings you want (Glove, Elmo, Bert, custom etc.)
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

bert = BertEmbeddings.pretrained('bert_base_cased', 'en')\
    .setInputCols(["sentence",'token'])\
    .setOutputCol("bert")\
    .setCaseSensitive(False)

loaded_ner_model = NerDLModel.load("NER_bert_20200221")\
   .setInputCols(["sentence", "token", "bert"])\
   .setOutputCol("ner")


ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_span')

custom_ner_pipeline = Pipeline(stages=[
      document_assembler, 
      sentence,
      tokenizer,
      bert,
      loaded_ner_model,
      ner_converter
 ])

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


2023-01-20 09:54:51.654497: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

In [11]:
from sparknlp.base import LightPipeline

text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = custom_ner_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['ner'])

{'document': ['navigate to kitchen'], 'ner_span': ['navigate to kitchen'], 'token': ['navigate', 'to', 'kitchen'], 'ner': ['Act', '0', 'Loc'], 'bert': ['navigate', 'to', 'kitchen'], 'sentence': ['navigate to kitchen']}
['Act', '0', 'Loc']


In [12]:
light.annotate("Navigate to kitchen")

{'document': ['Navigate to kitchen'],
 'ner_span': ['Navigate to kitchen'],
 'token': ['Navigate', 'to', 'kitchen'],
 'ner': ['Act', '0', 'Loc'],
 'bert': ['navigate', 'to', 'kitchen'],
 'sentence': ['Navigate to kitchen']}

In [ ]:
text = input("Enter Testing Text\n")
while(text != "exit"):
    results = light.annotate(text)
    print(results['ner'])
    text = input("Enter New Text\n")

Enter Testing Text
laundry
['0']
Enter New Text
entryway
['Loc']
Enter New Text
lobby
['Loc']
Enter New Text
laundry
['0']
Enter New Text
shut down
['Act', 'Act']
Enter New Text
resume
['Act']
Enter New Text
cabinet
['Obj']
Enter New Text
sofa
['Obj']
Enter New Text
open cabinet
['Act', 'Obj']
Enter New Text
kiss
['Act']
Enter New Text
gwen
['Act']
Enter New Text
weed
['Act']
Enter New Text
smoke
['Act']
Enter New Text
smoke weed
['Act', 'Obj']
Enter New Text
closet
['Loc']
Enter New Text
parlor
['Loc']
Enter New Text
attict
['Loc']
Enter New Text
den
['Loc']
Enter New Text
coffe
['Obj']
Enter New Text
coffee
['Obj']
Enter New Text
I want to make coffe
['0', 'Rqt', '0', 'Act', 'Obj']
Enter New Text
I want to make coffee
['0', 'Rqt', '0', 'Act', 'Obj']
Enter New Text
laundryroom
['Loc']
Enter New Text
living room
['0', '0']
Enter New Text
room
['0']
Enter New Text
i want to go to the room
['0', 'Rqt', '0', 'Act', '0', '0', '0']
Enter New Text
how to get to my room
['Act', '0', 'Act', '0